In [2]:
"""
Enhanced LSTM Movie Recommendation Model v2

Improvements over v1:
1. Larger embedding and hidden dimensions
2. Bidirectional LSTM (sees sequence from both directions)
3. Attention mechanism (focuses on important movies in history)
4. Better regularization and training schedule
5. Early stopping to prevent overfitting

Author: Bhaagwat
Project: Context Learning Movie Recommendation
"""

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
from tqdm import tqdm
from pathlib import Path
import math

# ============================================================
# GOOGLE DRIVE SETUP
# ============================================================
def setup_google_drive():
    """Mount Google Drive for saving models."""
    try:
        from google.colab import drive
        drive.mount('/content/drive')

        # Create project folder if it doesn't exist
        drive_path = Path('/content/drive/MyDrive/MovieRecommendation')
        drive_path.mkdir(parents=True, exist_ok=True)

        print(f"✓ Google Drive mounted!")
        print(f"✓ Models will be saved to: {drive_path}")
        return drive_path
    except Exception as e:
        print(f"⚠ Could not mount Google Drive: {e}")
        print("  Models will be saved locally instead.")
        return Path('.')


class Attention(nn.Module):
    """
    Simple attention mechanism.
    Learns which movies in the sequence are most important for prediction.
    """
    def __init__(self, hidden_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.Tanh(),
            nn.Linear(hidden_dim // 2, 1)
        )

    def forward(self, lstm_output):
        # lstm_output: [batch, seq_len, hidden_dim]
        attention_weights = self.attention(lstm_output)  # [batch, seq_len, 1]
        attention_weights = torch.softmax(attention_weights, dim=1)

        # Weighted sum of LSTM outputs
        context = torch.sum(lstm_output * attention_weights, dim=1)  # [batch, hidden_dim]
        return context, attention_weights


class MovieLSTMv2(nn.Module):
    """
    Enhanced LSTM with:
    - Larger dimensions
    - Bidirectional processing
    - Attention mechanism
    - Layer normalization
    """

    def __init__(self, vocab_size: int, embedding_dim: int = 256,
                 hidden_dim: int = 512, num_layers: int = 2, dropout: float = 0.4):
        super().__init__()

        self.vocab_size = vocab_size
        self.hidden_dim = hidden_dim

        # Embedding with dropout
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embed_dropout = nn.Dropout(dropout)

        # Bidirectional LSTM
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True  # Key change!
        )

        # Layer normalization
        self.layer_norm = nn.LayerNorm(hidden_dim * 2)  # *2 for bidirectional

        # Attention over sequence
        self.attention = Attention(hidden_dim * 2)

        # Output layers (bigger capacity)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, vocab_size)
        )

    def forward(self, x):
        """
        Args:
            x: [batch_size, seq_len] movie indices
        Returns:
            logits: [batch_size, vocab_size] scores for each movie
        """
        # Embed: [batch, seq_len, embedding_dim]
        embedded = self.embedding(x)
        embedded = self.embed_dropout(embedded)

        # Bidirectional LSTM: [batch, seq_len, hidden_dim * 2]
        lstm_out, _ = self.lstm(embedded)
        lstm_out = self.layer_norm(lstm_out)

        # Attention: [batch, hidden_dim * 2]
        context, _ = self.attention(lstm_out)

        # Predict: [batch, vocab_size]
        logits = self.fc(context)

        return logits


def load_data(data_path: str):
    """Load the prepared .pth dataset."""
    print(f"Loading data from {data_path}...")
    data = torch.load(data_path)

    context = data['context']
    target = data['target']
    vocab_size = data['vocab_size']

    print(f"  Samples: {len(target):,}")
    print(f"  Vocab size: {vocab_size:,}")
    print(f"  Context window: {context.shape[1]}")

    return context, target, vocab_size, data


def create_dataloaders(context, target, batch_size=256, val_split=0.1, test_split=0.1):
    """Create train/val/test dataloaders."""
    dataset = TensorDataset(context, target)

    # Split into train/val/test
    total = len(dataset)
    test_size = int(total * test_split)
    val_size = int(total * val_split)
    train_size = total - val_size - test_size

    train_dataset, val_dataset, test_dataset = random_split(
        dataset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    print(f"  Train: {len(train_dataset):,}")
    print(f"  Val: {len(val_dataset):,}")
    print(f"  Test: {len(test_dataset):,}")

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader


def calculate_topk_accuracy(logits, targets, k=10):
    """Calculate top-k accuracy."""
    _, top_indices = logits.topk(k, dim=1)
    correct = (top_indices == targets.unsqueeze(1)).any(dim=1)
    return correct.float().mean().item()


def train_epoch(model, train_loader, optimizer, criterion, device, clip_grad=1.0):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    total_top1, total_top5, total_top10, total_top20 = 0, 0, 0, 0
    num_batches = 0

    pbar = tqdm(train_loader, desc="Training")
    for context, target in pbar:
        context, target = context.to(device), target.to(device)

        optimizer.zero_grad()
        logits = model(context)
        loss = criterion(logits, target)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)
        optimizer.step()

        total_loss += loss.item()
        total_top1 += calculate_topk_accuracy(logits, target, k=1)
        total_top5 += calculate_topk_accuracy(logits, target, k=5)
        total_top10 += calculate_topk_accuracy(logits, target, k=10)
        total_top20 += calculate_topk_accuracy(logits, target, k=20)
        num_batches += 1

        pbar.set_postfix({
            'loss': f'{loss.item():.3f}',
            'top10': f'{total_top10/num_batches:.3f}'
        })

    return {
        'loss': total_loss / num_batches,
        'top1': total_top1 / num_batches,
        'top5': total_top5 / num_batches,
        'top10': total_top10 / num_batches,
        'top20': total_top20 / num_batches
    }


def evaluate(model, loader, criterion, device, desc="Evaluating"):
    """Evaluate the model."""
    model.eval()
    total_loss = 0
    total_top1, total_top5, total_top10, total_top20 = 0, 0, 0, 0
    num_batches = 0

    with torch.no_grad():
        for context, target in tqdm(loader, desc=desc):
            context, target = context.to(device), target.to(device)

            logits = model(context)
            loss = criterion(logits, target)

            total_loss += loss.item()
            total_top1 += calculate_topk_accuracy(logits, target, k=1)
            total_top5 += calculate_topk_accuracy(logits, target, k=5)
            total_top10 += calculate_topk_accuracy(logits, target, k=10)
            total_top20 += calculate_topk_accuracy(logits, target, k=20)
            num_batches += 1

    return {
        'loss': total_loss / num_batches,
        'top1': total_top1 / num_batches,
        'top5': total_top5 / num_batches,
        'top10': total_top10 / num_batches,
        'top20': total_top20 / num_batches
    }


def print_metrics(metrics, prefix=""):
    """Pretty print metrics."""
    print(f"{prefix}Loss: {metrics['loss']:.4f} | "
          f"Top-1: {metrics['top1']*100:.2f}% | "
          f"Top-5: {metrics['top5']*100:.2f}% | "
          f"Top-10: {metrics['top10']*100:.2f}% | "
          f"Top-20: {metrics['top20']*100:.2f}%")


class EarlyStopping:
    """Early stopping to prevent overfitting."""
    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.should_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.should_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

        return self.should_stop


def predict_next_movies(model, movie_sequence, movie_to_idx, idx_to_movie,
                        movie_info, device, top_k=10):
    """Predict next movies given a sequence."""
    model.eval()

    # Convert to indices
    indices = [movie_to_idx.get(m, 0) for m in movie_sequence]
    context = torch.tensor([indices], dtype=torch.long).to(device)

    with torch.no_grad():
        logits = model(context)
        probs = torch.softmax(logits, dim=1)
        top_probs, top_indices = probs.topk(top_k, dim=1)

    recommendations = []
    for prob, idx in zip(top_probs[0].cpu().numpy(), top_indices[0].cpu().numpy()):
        movie_id = idx_to_movie.get(idx)
        if movie_id and movie_id in movie_info:
            recommendations.append({
                'movieId': movie_id,
                'title': movie_info[movie_id]['title'],
                'genres': movie_info[movie_id]['genres'],
                'confidence': float(prob)
            })

    return recommendations


def main():
    # ============================================================
    # MOUNT GOOGLE DRIVE
    # ============================================================
    drive_path = setup_google_drive()

    # ============================================================
    # CONFIGURATION
    # ============================================================
    class Config:
        # Data (local)
        data = './movie_sequences_v3.pth'

        # Output (Google Drive)
        output = str(drive_path / 'movie_lstm_v3_trained.pth')
        checkpoint_dir = drive_path / 'checkpoints'

        # Model architecture (BIGGER)
        embedding_dim = 256         # Was 128
        hidden_dim = 512            # Was 256
        num_layers = 2
        dropout = 0.4               # Slightly more dropout for bigger model

        # Training
        batch_size = 256            # Smaller batch for better gradients
        epochs = 50                 # More epochs
        lr = 0.001
        weight_decay = 1e-5         # L2 regularization

        # Early stopping
        patience = 7                # Stop if no improvement for 7 epochs

        # Checkpointing
        save_every_n_epochs = 5     # Save checkpoint every N epochs

        # Device
        device = 'auto'

    config = Config()

    # Create checkpoint directory
    config.checkpoint_dir.mkdir(parents=True, exist_ok=True)

    # Device
    if config.device == 'auto':
        if torch.cuda.is_available():
            device = torch.device('cuda')
        elif torch.backends.mps.is_available():
            device = torch.device('mps')
        else:
            device = torch.device('cpu')
    else:
        device = torch.device(config.device)

    print("\n" + "="*70)
    print("ENHANCED MOVIE LSTM v2 TRAINING")
    print("="*70)
    print(f"Device: {device}")
    print(f"Embedding dim: {config.embedding_dim}")
    print(f"Hidden dim: {config.hidden_dim}")
    print(f"LSTM layers: {config.num_layers} (bidirectional)")
    print(f"Dropout: {config.dropout}")
    print(f"Batch size: {config.batch_size}")
    print(f"Max epochs: {config.epochs}")
    print(f"Early stopping patience: {config.patience}")
    print("="*70 + "\n")

    # Load data
    context, target, vocab_size, full_data = load_data(config.data)

    # Create dataloaders
    train_loader, val_loader, test_loader = create_dataloaders(
        context, target, config.batch_size
    )

    # Create model
    model = MovieLSTMv2(
        vocab_size=vocab_size,
        embedding_dim=config.embedding_dim,
        hidden_dim=config.hidden_dim,
        num_layers=config.num_layers,
        dropout=config.dropout
    ).to(device)

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model parameters: {total_params:,} ({trainable_params:,} trainable)\n")

    # Loss, optimizer, scheduler
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Label smoothing helps!
    optimizer = optim.AdamW(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)

    # Cosine annealing with warm restarts
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=10, T_mult=2, eta_min=1e-6
    )

    early_stopping = EarlyStopping(patience=config.patience)

    # Training loop
    best_val_top10 = 0
    best_metrics = None

    print("-"*70)
    for epoch in range(config.epochs):
        current_lr = optimizer.param_groups[0]['lr']
        print(f"\nEpoch {epoch+1}/{config.epochs} (lr: {current_lr:.6f})")

        # Train
        train_metrics = train_epoch(model, train_loader, optimizer, criterion, device)
        print_metrics(train_metrics, "  Train: ")

        # Validate
        val_metrics = evaluate(model, val_loader, criterion, device, "Validating")
        print_metrics(val_metrics, "  Val:   ")

        # Scheduler step
        scheduler.step()

        # Save best model (by top-10 accuracy, not loss)
        if val_metrics['top10'] > best_val_top10:
            best_val_top10 = val_metrics['top10']
            best_metrics = val_metrics

            save_dict = {
                'model_state_dict': model.state_dict(),
                'vocab_size': vocab_size,
                'embedding_dim': config.embedding_dim,
                'hidden_dim': config.hidden_dim,
                'num_layers': config.num_layers,
                'dropout': config.dropout,
                'movie_to_idx': full_data['movie_to_idx'],
                'idx_to_movie': full_data['idx_to_movie'],
                'movie_info': full_data['movie_info'],
                'context_size': full_data['context_size'],
                'best_metrics': best_metrics,
                'model_version': 'v2_bidirectional_attention'
            }
            torch.save(save_dict, config.output)
            print(f"  ✓ New best! Top-10: {best_val_top10*100:.2f}% → Saved to Google Drive")

        # Save periodic checkpoint
        if (epoch + 1) % config.save_every_n_epochs == 0:
            checkpoint_path = config.checkpoint_dir / f'checkpoint_epoch_{epoch+1}.pth'
            checkpoint_dict = {
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'val_metrics': val_metrics,
                'best_val_top10': best_val_top10,
                'vocab_size': vocab_size,
                'config': {
                    'embedding_dim': config.embedding_dim,
                    'hidden_dim': config.hidden_dim,
                    'num_layers': config.num_layers,
                    'dropout': config.dropout
                }
            }
            torch.save(checkpoint_dict, checkpoint_path)
            print(f"  💾 Checkpoint saved: {checkpoint_path}")

        # Early stopping check
        if early_stopping(val_metrics['loss']):
            print(f"\n  Early stopping triggered after {epoch+1} epochs")
            break

    # Final evaluation on test set
    print("\n" + "="*70)
    print("FINAL EVALUATION ON TEST SET")
    print("="*70)

    # Load best model
    checkpoint = torch.load(config.output)
    model.load_state_dict(checkpoint['model_state_dict'])

    test_metrics = evaluate(model, test_loader, criterion, device, "Testing")
    print_metrics(test_metrics, "Test: ")

    print("\n" + "="*70)
    print("TRAINING COMPLETE")
    print("="*70)
    print(f"Best validation metrics:")
    print(f"  Top-1:  {best_metrics['top1']*100:.2f}%")
    print(f"  Top-5:  {best_metrics['top5']*100:.2f}%")
    print(f"  Top-10: {best_metrics['top10']*100:.2f}%")
    print(f"  Top-20: {best_metrics['top20']*100:.2f}%")
    print(f"\n📁 Files saved to Google Drive:")
    print(f"  Best model: {config.output}")
    print(f"  Checkpoints: {config.checkpoint_dir}")
    print("="*70)

    # Demo predictions
    print("\n" + "-"*70)
    print("DEMO PREDICTIONS")
    print("-"*70)

    # Get a few samples
    context_size = full_data['context_size']
    for i in range(3):
        sample_idx = i * 1000
        sample_context = context[sample_idx].tolist()
        sample_movies = [full_data['idx_to_movie'].get(idx) for idx in sample_context]
        actual_target = full_data['idx_to_movie'].get(target[sample_idx].item())

        print(f"\nExample {i+1}:")
        print("  Watch history:")
        for j, movie_id in enumerate(sample_movies):
            if movie_id and movie_id in full_data['movie_info']:
                title = full_data['movie_info'][movie_id]['title']
                print(f"    {j+1}. {title}")

        # Actual next movie
        if actual_target and actual_target in full_data['movie_info']:
            actual_title = full_data['movie_info'][actual_target]['title']
            print(f"\n  Actual next movie: {actual_title}")

        # Predictions
        print("  Predicted:")
        recs = predict_next_movies(
            model, sample_movies,
            full_data['movie_to_idx'],
            full_data['idx_to_movie'],
            full_data['movie_info'],
            device, top_k=5
        )

        for j, rec in enumerate(recs):
            match = "✓" if rec['movieId'] == actual_target else " "
            print(f"    {j+1}. {match} {rec['title']} ({rec['confidence']*100:.1f}%)")

    print("="*70 + "\n")


if __name__ == '__main__':
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Google Drive mounted!
✓ Models will be saved to: /content/drive/MyDrive/MovieRecommendation

ENHANCED MOVIE LSTM v2 TRAINING
Device: cuda
Embedding dim: 256
Hidden dim: 512
LSTM layers: 2 (bidirectional)
Dropout: 0.4
Batch size: 256
Max epochs: 50
Early stopping patience: 7

Loading data from ./movie_sequences_v3.pth...
  Samples: 1,521,672
  Vocab size: 5,480
  Context window: 10
  Train: 1,217,338
  Val: 152,167
  Test: 152,167
Model parameters: 14,984,553 (14,984,553 trainable)

----------------------------------------------------------------------

Epoch 1/50 (lr: 0.001000)


Training: 100%|██████████| 4756/4756 [04:25<00:00, 17.90it/s, loss=6.958, top10=0.065]


  Train: Loss: 7.3347 | Top-1: 0.85% | Top-5: 3.60% | Top-10: 6.49% | Top-20: 11.40%


Validating: 100%|██████████| 595/595 [00:11<00:00, 53.83it/s]


  Val:   Loss: 7.0038 | Top-1: 1.27% | Top-5: 5.51% | Top-10: 9.75% | Top-20: 16.17%
  ✓ New best! Top-10: 9.75% → Saved to Google Drive

Epoch 2/50 (lr: 0.000976)


...

  Train: Loss: 6.5032 | Top-1: 2.97% | Top-5: 10.38% | Top-10: 16.84% | Top-20: 26.14%


Validating: 100%|██████████| 595/595 [00:11<00:00, 51.61it/s]


  Val:   Loss: 6.5325 | Top-1: 2.98% | Top-5: 10.26% | Top-10: 16.70% | Top-20: 25.80%
  💾 Checkpoint saved: /content/drive/MyDrive/MovieRecommendation/checkpoints/checkpoint_epoch_35.pth

  Early stopping triggered after 35 epochs

FINAL EVALUATION ON TEST SET


Testing: 100%|██████████| 595/595 [00:11<00:00, 50.75it/s]


Test: Loss: 6.5220 | Top-1: 2.98% | Top-5: 10.37% | Top-10: 16.84% | Top-20: 26.06%

TRAINING COMPLETE
Best validation metrics:
  Top-1:  3.09%
  Top-5:  10.44%
  Top-10: 16.78%
  Top-20: 25.89%

📁 Files saved to Google Drive:
  Best model: /content/drive/MyDrive/MovieRecommendation/movie_lstm_v3_trained.pth
  Checkpoints: /content/drive/MyDrive/MovieRecommendation/checkpoints

----------------------------------------------------------------------
DEMO PREDICTIONS
----------------------------------------------------------------------

Example 1:
  Watch history:
    1. American Pie 2 (2001)
    2. Back to the Future (1985)
    3. Groundhog Day (1993)
    4. Weird Science (1985)
    5. Rocky IV (1985)
    6. Braveheart (1995)
    7. Good Will Hunting (1997)
    8. Sixth Sense, The (1999)
    9. Ferris Bueller's Day Off (1986)
    10. Game, The (1997)

  Actual next movie: Breakfast Club, The (1985)
  Predicted:
    1.   Good Will Hunting (1997) (1.4%)
    2.   Sixth Sense, The (1999) (1